## Inicio del Proceso ETL

### Orden de Ejecución
Este archivo es el primero en la secuencia de ejecución de los notebooks del proceso ETL para **Google**. Asegúrate de seguir el orden establecido para garantizar la correcta integración y procesamiento de los datos.

### Fuentes de Datos
Los datos se obtienen de archivos JSON que contienen información sobre negocios de Google Maps. Cada archivo representa un conjunto de datos relacionado con diferentes aspectos de los negocios, como nombres, direcciones, calificaciones y servicios ofrecidos.

### Estructura de Datos
Se ha realizado el proceso ETL en base al diagrama de entidad-relación (E/R) creado anteriormente, siguiendo el orden de tablas, tipos de datos e identificadores únicos. Esto asegura una comprensión clara de cómo se relacionan los datos entre sí y proporciona una estructura coherente para el diseño del esquema en la base de datos.

### Tipo de Datos
Los datos son considerados **Big Data** debido a su volumen y diversidad. Cada archivo JSON es tratado como un conjunto de datos que se procesará en el flujo ETL.

### Almacenamiento de Archivos
Los archivos JSON se encuentran en la siguiente ruta de Google Drive:
https://drive.google.com/drive/folders/1Wf7YkxA0aHI3GpoHc9Nh8_scf5BbD4DA



In [1]:
# Librerias Necesarias 
import pandas as pd
import os

In [ ]:
# Los 11 archivos Jsons estaticos de Google, los juntamos en un solo archivo.
list_sitios_path = [
    'datasets/google/sitios/1.json',
    'datasets/google/sitios/2.json',
    'datasets/google/sitios/3.json',
    'datasets/google/sitios/4.json',
    'datasets/google/sitios/5.json',
    'datasets/google/sitios/6.json',
    'datasets/google/sitios/7.json',
    'datasets/google/sitios/8.json',
    'datasets/google/sitios/9.json',
    'datasets/google/sitios/10.json',
    'datasets/google/sitios/11.json'
]

for i in range(11):
    df = pd.read_json(list_sitios_path[i], lines=True)
    if i == 0:
        g_sitios = df.copy()
        del df
    else:
        g_sitios = pd.concat([g_sitios, df], axis=0, ignore_index=True)
        del df

# Eliminamos filas que repitan la clave primaria y unica        
g_sitios = g_sitios.drop_duplicates(subset=['gmap_id'])

In [ ]:
#  Guardamos el conjunto de datos en Parquet para optimizacion de espacio, con los datos originales para un respaldo inicial.
g_sitios.to_parquet('datasets/google/sitios/sitios_google_columnas_originales.parquet')

In [ ]:
# Funcion para descomponer la Lista de los Horarios
def retornar_horario(campo:list[str], dia:str) -> str:
    """ Función de apoyo a los campos que representan el horario de
                         cada dia: monday, tuesday....sunday

    Args:
        campo (list[list]): lista de la columna `hours` 
        dia (str): dia de la semana a consultar en campo

    Returns:
        str: Horario del dia especifico que se consulta
    """
    try:
        for i in range(7):
            if campo[i][0].lower() != dia.lower():
                continue
            buscado = campo[i][1]
            break
        return buscado
    except:
        return 'No Disponible'

In [ ]:
# Se Crea las Columnas de los Horarios

g_sitios['monday'] = g_sitios['hours'].apply(lambda x: retornar_horario(x, dia='Monday'))

g_sitios['tuesday'] = g_sitios['hours'].apply(lambda x: retornar_horario(x, dia='Tuesday'))

g_sitios['wednesday'] = g_sitios['hours'].apply(lambda x: retornar_horario(x, dia='Wednesday'))

g_sitios['thursday'] = g_sitios['hours'].apply(lambda x: retornar_horario(x, dia='Thursday'))

g_sitios['friday'] = g_sitios['hours'].apply(lambda x: retornar_horario(x, dia='Friday'))

g_sitios['saturday'] = g_sitios['hours'].apply(lambda x: retornar_horario(x, dia='Saturday'))

g_sitios['sunday'] = g_sitios['hours'].apply(lambda x: retornar_horario(x, dia='Sunday'))

g_sitios.head()

In [ ]:
# Ahora podemos eliminar la columna horarios.
g_sitios.drop('hours', axis=1, inplace=True)

In [ ]:
# Creamos una nueva tabla relative_results
relative_results = g_sitios[['gmap_id', 'relative_results']]
# Desempaquetamos su información.
relative_results = relative_results.explode('relative_results')

In [ ]:
# Guaradamos la tabla desanidada que se conecta a travez de gmap_id con la tabla principal.
relative_results.to_parquet('datasets/google/sitios/relative_results.parquet')

### Listo el analisis inicial de la tabla principal, pasamos al Notebook 2, donde estara el desanidado y normalizacion de las demas tablas.